In [26]:
import os
import yaml
import openai

from langchain.agents import create_json_agent, AgentExecutor
from langchain.agents.agent_toolkits import JsonToolkit
from langchain.chains import LLMChain
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec

In [42]:
import json
import openai

# Function to parse the JSON file and format the data
def parse_json_file(file_path, delimiter='\n'):
    results = []
    with open(file_path, 'r') as file:
        data = json.load(file)
        for row in data:
            summary = row['summary']
            educator = row['Educator']
            results.append({
                'educator': educator,
                'summary': summary,
            })
    return results

# Prompt
delimiter = '\n'
system_message = f"""
You have a JSON file /summaries.json to parse and format and index. \
There are three key-value pairs. The keys are Summary:, Outcome:, and Follow-ups:. \
You should parse the JSON file and output a Python list of objects, \
and each object should have the following format:
    'educator': <the name of the educator who is the subject of the summary>,
    'summary': <the summary from the JSON file>,
    'outcome': <the outcome from the JSON file>,
    'follow_ups': <a list of follow-ups from the JSON file>

Only output the list of objects, with nothing else.
"""

# Parse JSON file and format the data
results = parse_json_file('chroma-langchain/summaries.json')

# Print the formatted data
for result in results:
    print(result)

# Output the list of objects as a JSON string
json_results = json.dumps(results)
print(json_results)

def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].text


FileNotFoundError: [Errno 2] No such file or directory: 'chroma-langchain/summaries.json'

In [21]:
# delimiter = '\n'
# system_message = f"""
# You have a csv file ./summaries.csv to parse and format and index. \
# there are three keys:value pairs. the keys are Summary:, Outcome: & Follow-ups: \
# You should parse the csv file and output a python list of objects, \
# and each object should have the following format:
#     'educator': <the name of the educator who is the subject of the summary>,
#     'summary': <the summary from the csv file>,
#     'outcome': <the outcome from the csv file>,
#     'follow_ups': <a list of follow-ups from the csv file>

# Only output the list of objects, with nothing else.
# """

# messages =  [  
# {'role':'system', 
#  'content': system_message},
# ] 
# category_and_product_response_1 = get_completion_from_messages(messages)
# print(category_and_product_response_1)

In [23]:
import csv

# Function to parse the CSV file and format the data
def parse_csv_file(file_path):
    results = []
    with open('summaries.csv', 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            educator = row['Summary:']
            summary = row['Outcome:']
            outcome = row['Follow-ups:']
            follow_ups = outcome.split(delimiter)
            results.append({
                'educator': educator,
                'summary': summary,
                'outcome': outcome,
                'follow_ups': follow_ups
            })
    return results

# Prompt
delimiter = '\n'
system_message = f"""
You have a csv file ./summaries.csv to parse and format and index. \
there are three keys:value pairs. the keys are Summary:, Outcome: & Follow-ups: \
You should parse the csv file and output a python list of objects, \
and each object should have the following format:
    'educator': <the name of the educator who is the subject of the summary>,
    'summary': <the summary from the csv file>,
    'outcome': <the outcome from the csv file>,
    'follow_ups': <a list of follow-ups from the csv file>

Only output the list of objects, with nothing else.
"""

# Parse CSV file and format the data
results = parse_csv_file('summaries.csv')

# Print the formatted data
for result in results:
    print(result)

# Output the list of objects as a JSON string
json_results = json.dumps(results)
print(json_results)

def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 #engine=aoai_deployment,
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model, # used by original OpenAI endpoint
        #engine=engine, # used by Azure OpenAI endpoint
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message["content"]

KeyError: 'Summary:'

In [ ]:
with open("openai_openapi.yml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec = JsonSpec(dict_=data, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)

json_agent_executor = create_json_agent(
    llm=OpenAI(temperature=0), toolkit=json_toolkit, verbose=True
)

In [ ]:
delimiter = '\n'
system_message = f"""
You have a csv file ./summaries.csv to parse and format and index. \
there are three keys:value pairs. the keys are Summary:, Outcome: & Follow-ups: \
You should parse the csv file and output a python list of objects, \
and each object should have the following format:
    'educator': <the name of the educator who is the subject of the summary>,
    'summary': <the summary from the csv file>,
    'outcome': <the outcome from the csv file>,
    'follow_ups': <a list of follow-ups from the csv file>

Only output the list of objects, with nothing else.
"""

messages =  [  
{'role':'system', 
 'content': system_message},
] 
category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

In [ ]:
json_agent_executor.run(
    "What are the required parameters in the request body to the /completions endpoint?"
)